In [17]:
# some ape notebook examples:
# curve steth analysis: https://github.com/fmrmf/hello-ape/blob/main/console/hello-ape.ipynb
# uniswap usdc/usdt 1bps pool analysis: https://gist.github.com/banteg/dcf6082ff7fc6ad51ce220146f29d9ff

from ape import accounts, chain, Contract, networks
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [2]:
networks.parse_network_choice('ethereum:mainnet:alchemy').__enter__()

<alchemy chain_id=1>

In [3]:
seaport = Contract("0x00000000006c3852cbEf3e08E8dF289169EdE581")
seaport.name()

'Seaport'

In [4]:
seaport.OrderFulfilled

OrderFulfilled(bytes32 orderHash, address indexed offerer, address indexed zone, address recipient, (uint8,address,uint256,uint256)[] offer, (uint8,address,uint256,uint256,address)[] consideration)

In [5]:
curr_block = chain.blocks[-1].number
blocks_1d = int(60*60*24/12)

In [6]:
start_block = curr_block - blocks_1d
start_block

15458752

In [7]:
%time recent_trades = seaport.OrderFulfilled.query("*", start_block=start_block)

INFO: Cache database has not been initialized
CPU times: user 35.9 s, sys: 1.74 s, total: 37.7 s
Wall time: 37.1 s


In [8]:
recent_trades.head()

,event_name,contract_address,event_arguments,transaction_hash,block_number,block_hash,log_index,transaction_index
0,OrderFulfilled,0x00000000006c3852cbEf3e08E8dF289169EdE581,{'offerer': '0xD3678aAd0c996231D6ED91014830259...,0x12d0f405533b9884f741e9746931c9143a64080ed1e7...,15458752,0xa681ca381a23b3dab2f99d61cc9d9d7e4f17cd408eaa...,129,104
1,OrderFulfilled,0x00000000006c3852cbEf3e08E8dF289169EdE581,{'offerer': '0x663B94FD8A80fFe66004799029FB518...,0x31dfb272682fbcd44e18cd0a29672bc66340e296e3ef...,15458752,0xa681ca381a23b3dab2f99d61cc9d9d7e4f17cd408eaa...,132,107
2,OrderFulfilled,0x00000000006c3852cbEf3e08E8dF289169EdE581,{'offerer': '0xD1C3c9cdBB335ec425ac44aA9975037...,0x00cd6e22779c0b7e29e459aa4ad645121e3a4e968da4...,15458752,0xa681ca381a23b3dab2f99d61cc9d9d7e4f17cd408eaa...,176,127
3,OrderFulfilled,0x00000000006c3852cbEf3e08E8dF289169EdE581,{'offerer': '0xB168f9575FA65fc25e21447228D994C...,0x3fbd9ea33e5e7039c62d6d66095d74b0321426d23ad7...,15458752,0xa681ca381a23b3dab2f99d61cc9d9d7e4f17cd408eaa...,180,132
4,OrderFulfilled,0x00000000006c3852cbEf3e08E8dF289169EdE581,{'offerer': '0x0F284580198bb8490F5cA28B999869e...,0xe13e091b53c9503475f0c7a10d77150a7d45e36a35e2...,15458753,0x9c33f21ccbd70807f1d23c4820a7d8d70326526aae25...,199,181


In [9]:
recent_trades["event_arguments"].iloc[-1]

{'offerer': '0xD8e34f3d8bCC1Ca84F307175203FEe03F7F2CB89',
 'zone': '0x004C00500000aD104D7DBd00e3ae0A5C00560C00',
 'orderHash': HexBytes('0x72c60417e55aefef7f31af797a525e29612cb04e96f6e184430b38f7b47e0154'),
 'recipient': '0xCEAb18Da9132ae7E712180B820f28C5ce6E76Ad7',
 'offer': ((2, '0xd93206bd0062cc054e397ecccdb8436c3fa5700e', 29032, 1),),
 'consideration': ((0,
   '0x0000000000000000000000000000000000000000',
   0,
   28050000000000000,
   '0xd8e34f3d8bcc1ca84f307175203fee03f7f2cb89'),
  (0,
   '0x0000000000000000000000000000000000000000',
   0,
   750000000000000,
   '0x0000a26b00c1f0df003000390027140000faa719'),
  (0,
   '0x0000000000000000000000000000000000000000',
   0,
   1200000000000000,
   '0xa57ec3a618b6969a6b1fec2ef4bf54b2afc5ca30'))}

In [10]:
"""
 *   event OrderFulfilled(
 *     bytes32 orderHash,
 *     address indexed offerer,
 *     address indexed zone,
 *     address fulfiller,
 *     SpentItem[] offer > (itemType, token, id, amount)
 *     ReceivedItem[] consideration > (itemType, token, id, amount, recipient)
 *   )
 *
"""

def parse_trades(trades):
    blocks = trades.iloc[::1000]["block_number"].to_list()
    timestamps = [chain.blocks[i].timestamp for i in blocks]
    trades["timestamp"] = pd.to_datetime(np.interp(trades["block_number"], blocks, timestamps) * 1e9)
    offers = [args["offer"] for args in recent_trades["event_arguments"]]
    receipts = [args["consideration"] for args in recent_trades["event_arguments"]]
    trades["offer_tokens"] = [offer[0][1] if len(offer) > 0 else 'null' for offer in offers]
    trades["receipt_tokens"] = [receipt[0][1] if len(receipt) > 0 else 'null' for receipt in receipts]
    return trades

In [11]:
%time parsed_trades = parse_trades(recent_trades)

CPU times: user 619 ms, sys: 33.7 ms, total: 653 ms
Wall time: 8.09 s


In [12]:
top_offered_tokens = pd.DataFrame(parsed_trades["offer_tokens"].value_counts().reset_index().values, 
                                  columns=["Offer token address", "Occurrences"])
top_received_tokens = pd.DataFrame(parsed_trades["receipt_tokens"].value_counts().reset_index().values,
                                  columns=["Receipt token address", "Occurrences"])


In [13]:
top_offered_tokens_contracts = [Contract(address) if address != 'null' else 'null' 
                                for address in top_offered_tokens["Offer token address"].head(5)]

In [14]:
top_received_tokens_contracts = [Contract(address) if address != 'null' 
                                and address != '0x0000000000000000000000000000000000000000'
                                else 'null'
                                for address in top_received_tokens["Receipt token address"].head(5)]

In [15]:
# most common offered token in seaport trades in the past day
top_offered_tokens_contracts

[<WETH9 0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2>,
 <Delusion 0x2c0d708C6E82DC6c427CE6A96BA5ff2EA09bE272>,
 <Namida 0x81cF764150665eE2F4FA62Ca1A9Ff4324b29ec2B>,
 <GangApeBlingClub 0xE0d2ACE5DEbC43d3e0e758577D111C4859DF6F7e>,
 <SharedStorefrontLazyMintAdapter 0xA604060890923Ff400e8c6f5290461A83AEDACec>]

In [16]:
# most common received token in seaport trades in the past day
top_received_tokens_contracts

['null',
 'null',
 <BaseRegistrarImplementation 0x57f1887a8BF19b14fC0dF6Fd9B2acc9Af147eA85>,
 <RedeemableToken 0xa49a0e5eF83cF89Ac8aae182f22E6464B229eFC8>,
 <Gala 0x15D4c048F83bd7e37d49eA4C83a07267Ec4203dA>]